# Complex Ambiguity Function (CAF)

In [ ]:
%matplotlib inline
from matplotlib import pylab
from matplotlib.pyplot import plot, ylim, xlabel, ylabel, grid, savefig, imshow, contour
import numpy as np
from matplotlib.pyplot import psd
pylab.rcParams['savefig.dpi'] = 300

In [ ]:
from caf_verilog import sim_helper as sh
from caf_verilog.caf import simple_caf
from gps_helper.prn import PRN
from sk_dsp_comm import sigsys as ss
from sk_dsp_comm import digitalcom as dc

In [ ]:
prn = PRN(10)
prn2 = PRN(20)
fs = 625e3
Ns = fs / 200e3
prn_seq = prn.prn_seq()
prn_seq = [*prn_seq, *prn_seq]
prn_seq2 = prn2.prn_seq()
prn_seq,b = ss.nrz_bits2(np.array(prn_seq), Ns)
prn_seq2,b2 = ss.nrz_bits2(np.array(prn_seq2), Ns)

In [ ]:
Px,f = psd(prn_seq, 2**12, Fs=fs)
plot(f, 10*np.log10(Px))

In [ ]:
f_size = 100
foas = np.arange(-f_size, f_size + 1) * 1000
foas, len(foas)

## CAF Module

In [ ]:
from caf_verilog.caf import CAF
from caf_verilog.sim_helper import sim_shift
from caf_verilog.xcorr import dot_xcorr

In [ ]:
len(prn_seq)

In [ ]:
center = 3000
corr_length = len(foas) - 1
shift = 15
ncorr = np.arange(0, corr_length * 2)
foa_offset = 16
theta_shift = np.exp(1j*2*np.pi*ncorr*(foas[foa_offset])/float(fs))
ref, rec = sim_shift(prn_seq, center, corr_length, shift=shift)
caf = CAF(ref, rec * theta_shift, foas, fs=fs, n_bits=8, ref_i_bits=8, rec_i_bits=8)
caf.gen_tb()

In [ ]:
ref = np.array(ref)
rec = np.array(rec)

In [ ]:
xcorr_res_shift = dot_xcorr(ref, rec * theta_shift)
xcorr_res = dot_xcorr(ref, rec)

In [ ]:
plot(rec.real)

In [ ]:
plot(np.abs(np.array(xcorr_res_shift)))
ylim([0, corr_length])
grid();
xlabel("Inverse Center Offset (Samples)")
savefig('caf_test_shift.png')

In [ ]:
am_pos = np.argmax(abs(np.array(xcorr_res_shift)))
am_pos, abs(np.array(xcorr_res))[am_pos]

In [ ]:
plot(abs(np.array(xcorr_res)))
ylim([0, corr_length])
grid();
am_pos = np.argmax(abs(np.array(xcorr_res)))
am_pos, abs(np.array(xcorr_res))[am_pos]
xlabel("Inverse Center Offset (Samples)")
savefig('caf_test_signal.png')

## CAF Surface Plot

In [ ]:
from caf_verilog.caf import simple_caf

In [ ]:
ref, rec = sim_shift(prn_seq, center, corr_length, shift=shift, freq_shift=foas[105], fs=fs, padding=True)
ref = np.array(ref)
rec = np.array(rec)

In [ ]:
caf_res, dt = simple_caf(ref, rec, foas=foas, fs=fs)

In [ ]:
caf_res_np = np.array(caf_res)

In [ ]:
imshow(caf_res_np, interpolation='none', extent=[-corr_length, corr_length, corr_length, -corr_length])
xlabel("N-th Sample")
ylabel("N-th Frequency Offset")
grid();

In [ ]:
contour(dt, foas / fs, caf_res_np, levels=int(np.log10(corr_length)))
ylabel("Offset in Hz")
xlabel("Offset in Time (s)")
grid();